In [1]:
!nvidia-smi

Thu Apr 11 11:57:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090         On | 00000000:23:00.0 Off |                  N/A |
| 30%   33C    P8               20W / 350W|      3MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from src.segmentation.framework_handlers import ultralytics_handler
ultralytics_handler.validate_version_and_gpu()

Ultralytics YOLOv8.0.229 🚀 Python-3.10.13 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Setup complete ✅ (64 CPUs, 503.6 GB RAM, 58.8/788.3 GB disk)


### Prepare Dataset
In this section, the dataset along with YOLO annotations is processed to conform to the input structure required by YOLO. The YOLO format necessitates organizing the dataset into two main directories, `images` and `labels`. Each of these directories is further divided into two subdirectories, `train` and `val`, for training and validation datasets respectively. This structure facilitates YOLO's training and validation processes.
WE can process the data by two options:
- Single SegmentsAI dataset
- Two SegmentsAI datasets, one for train and the other for test

In [ ]:
from src.annotation_handling import annotation_handler

Option 1: prepare data using a single dataset by splitting to train and test

In [ ]:
# prepare data from a single dataset to fine tune yolo v5
train_annotations_folder_name = "etaylor_cannabis_patches_test"
train_dataset_version = "v0.1"
saving_yaml_path = "/home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO"
yaml_file_path = annotation_handler.convert_coco_to_yolo_format_from_single_dataset(train_annotations_folder_name, train_dataset_version, saving_yaml_path, train_percentage=0.8)

Annotations /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test/annotations/export_coco-instance_etaylor_cannabis_patches_test_v0.1.json:  53%|█████▎    | 47/89 [00:00<00:00, 465.70it/s]

Annotations /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test/annotations/export_coco-instance_etaylor_cannabis_patches_test_v0.1.json: 100%|██████████| 89/89 [00:00<00:00, 479.47it/s]

COCO data converted successfully.
Results saved to /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test/annotations/yolo


The folder /home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test/annotations/yolo has been deleted.


Option 2: prepare dataset by using two different segments datasets, one for train and second for eval

In [ ]:
# prepare data from a train and test datasets to fine tune yolo v5
train_annotations_folder_name = "etaylor_cannabis_patches_train_26-04-2024_15-44-44"
train_dataset_version = "v0.1"
test_annotations_folder_name = "etaylor_cannabis_patches_test_26-04-2024_15-44-44"
test_dataset_version = "v0.1"
saving_yaml_path = "/home/etaylor/code_projects/thesis/src/segmentation/notebooks/ultralytics/yaml_files"
yaml_file_path = annotation_handler.convert_coco_to_yolo_format_from_train_test_datasets(train_annotations_folder_name, 
                                                                                        train_dataset_version, 
                                                                                        test_annotations_folder_name,
                                                                                        test_dataset_version,
                                                                                        saving_yaml_path, 
                                                                                        )

Annotations /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/annotations/export_coco-instance_etaylor_cannabis_patches_train_26-04-2024_15-44-44_v0.1.json: 100%|██████████| 369/369 [00:00<00:00, 389.55it/s]

COCO data converted successfully.
Results saved to /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/annotations/yolo



Annotations /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test_26-04-2024_15-44-44/annotations/export_coco-instance_etaylor_cannabis_patches_test_26-04-2024_15-44-44_v0.1.json: 100%|██████████| 106/106 [00:00<00:00, 418.79it/s]

COCO data converted successfully.
Results saved to /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test_26-04-2024_15-44-44/annotations/yolo


YAML configuration file saved to /home/etaylor/code_projects/thesis/src/segmentation/notebooks/ultralytics/yaml_files/yolo_dataset_config.yaml


### Ultralytics Settings

In here we can modify the settings of the yolo runs. 

In [ ]:
from ultralytics import settings
import config


runs_dir = "/home/etaylor/code_projects/thesis/src/segmentation/notebooks/ultralytics/runs"
# Update a setting
settings.update({'runs_dir': config.ULTRALYTICS_RUNS_DIR, 'weights_dir': config.ULTRALYTICS_WEIGHTS_DIR, 'datasets_dir': config.ULTRALYTICS_DATASETS_DIR})

# View all settings
print(settings)

{'settings_version': '0.0.4', 'datasets_dir': '/sise/home/etaylor/datasets', 'weights_dir': '/home/etaylor/weights', 'runs_dir': '/home/etaylor/code_projects/thesis/src/segmentation/notebooks/ultralytics/runs', 'uuid': '3eabad9ed9f85e3e726c43f30dfeac36b9c8e6cecf6546d5abc3435b89c57f67', 'sync': True, 'api_key': '', 'openai_api_key': '', 'clearml': True, 'comet': True, 'dvc': True, 'hub': True, 'mlflow': True, 'neptune': True, 'raytune': True, 'tensorboard': True, 'wandb': True}


In [4]:
# in case dataset already converted from coco to yolo format use manual path
yaml_file_path = ""

In [ ]:
import os

# currently available models for train and hparams fine tune
ultralytics_checkpoint_paths = "/home/etaylor/code_projects/thesis/checkpoints/ultralytics"

available_models_checkpoints = {
    "yolov5": "yolov5xu.pt",
    "yolov8": "yolov8x.pt",
    "yolov9": "yolov9c.pt",
    "rt_detr": "rtdetr-x.pt",
}
model_checkpoint_path = os.path.join(ultralytics_checkpoint_paths, available_models_checkpoints["yolov5"])

Here we can select between using the yolo models or the RT_DETR models

In [ ]:
from ultralytics import YOLO

model = YOLO(model_checkpoint_path)

If using the RT DETR models use the code below

In [ ]:
# from ultralytics import RTDETR

# model_checkpoint_path = os.path.join(ultralytics_checkpoint_paths, available_models_checkpoints["rt_detr"])
# model = RTDETR(model_checkpoint_path)

In [10]:
# regular train of the model
results = model.train(data=yaml_file_path, epochs=200, imgsz=512)

Ultralytics YOLOv8.0.229 🚀 Python-3.10.13 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO8/yaml_files/etaylor_cannabis_patches_all_11_4_24_v0.1_data.yaml, epochs=200, time=None, patience=50, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train31, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_

train: Scanning /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_all_11_4_24/annotations/yolo_split/labels/train... 185 images, 178 backgrounds, 0 corrupt: 100%|██████████| 363/363 [00:00<00:00, 3190.58it/s]

train: New cache created: /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_all_11_4_24/annotations/yolo_split/labels/train.cache



val: Scanning /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_all_11_4_24/annotations/yolo_split/labels/val... 41 images, 50 backgrounds, 0 corrupt: 100%|██████████| 91/91 [00:00<00:00, 2876.66it/s]

val: New cache created: /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_all_11_4_24/annotations/yolo_split/labels/val.cache


...

     13/200      8.84G      1.267      1.217      1.082         85        512: 100%|██████████| 23/23 [00:06<00:00,  3.57it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.74it/s]

                   all         91        719       0.46      0.584      0.469      0.263



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     14/200      8.76G      1.263      1.338      1.055        260        512:  48%|████▊     | 11/23 [00:03<00:03,  3.43it/s]


KeyboardInterrupt: 

In [7]:
valid_results = model.val()
print(valid_results)

Ultralytics YOLOv8.0.229 🚀 Python-3.10.13 torch-2.1.1+cu121 CUDA:0 (NVIDIA RTX 6000 Ada Generation, 48647MiB)


Model summary (fused): 268 layers, 68126457 parameters, 0 gradients, 257.4 GFLOPs


val: Scanning /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test/annotations/yolo_split/labels/val.cache... 16 images, 20 backgrounds, 0 corrupt: 100%|██████████| 36/36 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.46it/s]


                   all         36        167      0.624      0.573      0.645      0.423
                 clear         36         47      0.512       0.66      0.602      0.398
                cloudy         36        106      0.816      0.417      0.697      0.495
                 amber         36         14      0.543      0.643      0.635      0.377
Speed: 0.1ms preprocess, 52.4ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO8/runs/detect/train302
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fc7a4ad5030>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.0070

In [ ]:
from IPython.display import Image as show_image
show_image(filename="/sise/home/etaylor/runs/segment/train22/val_batch0_pred.jpg")

### Perform Infrence with the model

In [8]:
import cv2

image = cv2.imread("/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train/v0.1/IMG_2198_p12.png")

# Perform inference
results = model.predict(image)


0: 640x640 (no detections), 19.5ms
Speed: 12.6ms preprocess, 19.5ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
